In [ ]:
#========================================================================
# Copyright 2019 Science Technology Facilities Council
# Copyright 2019 University of Manchester
#
# This work is part of the Core Imaging Library developed by Science Technology	
# Facilities Council and University of Manchester
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# http://www.apache.org/licenses/LICENSE-2.0.txt
# 
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# 
#=========================================================================

## Reconstructing a dataset from DLS
This exercise will walk you through the reconstruction of a parallel beam 3D data set from Diamond Light Source. We'll follow from the previous 2D exercises to set up and use a 3D geomety.

**Learning objectives:**
1. Use CIL processors CentreOfTotation and Resizer to preprocess the data
2. Change the geometry to 3D
3. Apply the same reconstruction alorithms to real data

**ToDo: reading in a dataset and manpulating the data in to the expected form**
**ToDo: right out new data??**

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from ccpi.framework import ImageData, ImageGeometry
from ccpi.framework import AcquisitionGeometry, AcquisitionData

from ccpi.optimisation.algorithms import CGLS, SIRT
from ccpi.optimisation.functions import Norm2Sq, L1Norm
from ccpi.optimisation.operators import BlockOperator, Gradient, Identity
from ccpi.framework import BlockDataContainer

from ccpi.processors import Resizer, CenterOfRotationFinder

from ccpi.io import NEXUSDataReader
from ccpi.astra.operators import AstraProjectorSimple , AstraProjector3DSimple
from ccpi.astra.processors import FBP

# All external imports
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import scipy
from utilities import islicer, link_islicer
from utilities import plotter2D


### Read in the dataset

Use the NEXUS data reader to read in a dataset from the Diamond Light Source. The data reader creates the AquisitionData object for you with the geometry specified in the file.

CIL also provides a reader for Nikon datasets `NikonDataReader()`.

In [ ]:
## Set up a reader object pointing to the Nexus data set
path = os.path.join(sys.prefix, 'share','ccpi','24737_fd_normalised.nxs')
myreader = NEXUSDataReader(nexus_file=path)
data = myreader.load_data()

data.fill(np.random.poisson(500*data.as_array())/500)

#Convert the data from intensity to attenuation by taking the negative log
data.log(out=data)
data *= -1

The NEXUSDataReader output is either an ImageData object or an AcquisitonData Object. This is decided by the fields present in the dataset.

In [ ]:
print(type(data))
print(data)

We have created an AcquisitionData object from the input file. We can see the raw data has 3-axes where 'vertical' and 'horizontal' describe the output of the detector, with 'angle' giving the rotation of the object.

We can look closer at the data we have read in, and have a look at the data.

In [ ]:
print(data.geometry)

islicer(data, direction=0, minmax=(0,3))

### Centre of Rotation

The Centre of Rotation is the projection of the rotate axis on to the detector. The reconstruction assumes this is in the centre of the detector, and it being offset introduces blutting and artefacts.

**ToDo: show geometry** 

We need to reproccess the data by either padding or cropping the projections to achieve this.

The code below reconstucts one slice of the data. **ToDo Exercise change range to find the best looking reconstruction**

In [ ]:
title = []
results = []

slice_num = 68
data_transposed =  data.subset(dimensions=['vertical','angle','horizontal']).subset(vertical=slice_num)
data_transposed.geometry.angles = data.geometry.angles * np.pi /180.

# Create Acquisition Geometry
ag = data_transposed.geometry.clone()

# Create Image Geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z= 1 )

start = 4
step = 1

for n in range(0,6):
    shift = start + n * step  
    data_cor = ag.allocate()
    
    scipy.ndimage.interpolation.shift(data_transposed.as_array(), (0,-shift), output = data_cor.as_array(), order=1,mode='nearest')
    
    #Initialise the processor
    fbp = FBP(ig, ag, device='gpu')
    fbp.set_input(data_cor)
    FBP_output = fbp.get_output()  

    title.append("CoR = %s pixels" % shift)
    results.append(FBP_output.as_array()[:,:])

plotter2D(results,title,fix_range=True)

### Use processors to pre-proccess the data

CIL gives you access to some commonly needed data processors including:
- `Normalizer()` normalises AcquisitionData based on the instrument reading with and without incident photons or neutrons
- `Resizer()` allows you to crop or bin the data in any dimension
- `CenterOfRotationFinder()` finds the center of rotation in a parallel beam dataset (credit: Nghia Vo)

The processors are called in the following way:<br>
>processor_instance = Processor(set_up_parameters)<br>
>processor_instance.set_input(data_in)<br>
>data_out = processor_instance.get_output()<br>

#### Use CenterOfRotationFinder()

We can use `CentreOfRotationFinder()` to locate the Centre of Rotation in a parallel beam dataset. The output is in pixels at the detector.

**ToDo: show diagram** 

**ToDo: fix CoR to work on transposed data**

**ToDo: exercise??**

In [ ]:
# initialise the processsor
cor = CenterOfRotationFinder()

In [ ]:
# set the input data
cor.set_input(data)

In [ ]:
# get the output data
center_of_rotation = cor.get_output()
print("Centre of rotation at x = ", center_of_rotation)

In [ ]:
shift = (center_of_rotation - data.shape[2]/2)
print("Centre of rotation - detector centre = ", shift, " pixels")

Does this agree with the reconstructions above? **ToDo: link**

#### Correct the acquisition data for the centre of rotation offset

In [ ]:
#allocate the memory
data_centred = data.geometry.allocate()

#shift = 0
#use scipy to do a translation and interpolation of each projection image
shifted = scipy.ndimage.interpolation.shift(data.as_array(), (0,0,-shift), order=1,mode='nearest')
data_centred.fill(shifted)


In [ ]:
islicer(data_centred, direction=0, minmax=(0,3))

**ToDo: exercise??**
Run the CentreOfRotationFinder() over the centred data set and convince yourself it's now close to the centre of the detector.

In [ ]:
# initialise the processsor
cor = CenterOfRotationFinder()

# set the input data
cor.set_input(data_centred)

# get the output data
center_of_rotation = cor.get_output()

print("Centre of rotation at x = ", center_of_rotation)
shift = (center_of_rotation - data.shape[2]/2)
print("Centre of rotation - detector centre = ", shift, " pixels")

### Use Resizer()
`Resizer(roi, binning)`

To crop the data pass the optional parameter `roi` (region of interest). This is passed as a list where each element defines the behaviour in one dimension. To crop along an axis pass a tuple of the start and end coordinates of the crop, otherwise pass -1.

To bin the data in any dimension pass an optional paramer `binning`. This is a list with the number of pixels to bin in each dimension.

In [ ]:
print(data_centred)

In [ ]:
#define the region of interest
roi_crop = [-1,-1,-1]
bins = [1, 1, 1]

In [ ]:
#initialise the processsor
resizer = Resizer(roi=roi_crop, binning=bins)

In [ ]:
#set the input data
resizer.set_input(data_centred)

In [ ]:
#get the output data
data_reduced = resizer.get_output()

In [ ]:
#Note the acquistion geometry has also been modified
print(data_reduced)
islicer(data_centred, direction=0)
islicer(data_reduced, direction=0)

### Set up the data ready for ASTRA

**ToDo: Data sets come in different forms so we need to make some changes before reconstruction** 

ASTRA expects the data in the order `['vertical','angle','horizontal']` so we need to transpose the dataset.

We can use `AcquisitionData.subset()` which returns a subset of the AcquisitionData and regenerates the geometry.

In [ ]:
data_processed = data_reduced.subset(dimensions=['vertical','angle','horizontal'])
print(data_reduced)
print(data_processed)

ASTRA also requires the of projection angles to be in radians. Diamond outputs them in degrees so we need to convert them

In [ ]:
#convert the angles to radians
if data_processed.geometry.angle_unit == 'degree':
    data_processed.geometry.angle_unit = 'radian'
    data_processed.geometry.angles = data_reduced.geometry.angles * np.pi /180.

In [ ]:
#look at the acquisition data
islicer(data_processed, direction=1)

### Define the Geometry

**ToDo: add geometry diagrame here**

#### Acquistion geometry
In the 2D example we used:<br>
`ag = AcquisitionGeometry(geom_type='parallel', dimension='2D', angles=angles, pixel_num_h=number_pixels_x)`<br>

For 3D we need to change the dimension description to ` dimension='3D'`, and pass the number of vertical pixels as `pixel_num_v`<br>

However we've been using the acquistion geometry throughout this notebook so let's just clone the version we've already set up.

#### Image geometry
In the 2D example we used:<br>
`ig = ImageGeometry(voxel_num_x = num_voxels_xy, voxel_num_y = num_voxels_xy)`

For ad 3D reconstruction we also need to pass the number of voxels we want in the $z$-direction as `voxel_num_z`

In [ ]:
# Create Acquisition Geometry
ag = data_processed.geometry.clone()

In [ ]:
# Create Image Geometry
ig = ImageGeometry(voxel_num_x=ag.pixel_num_h,
                   voxel_num_y=ag.pixel_num_h, 
                   voxel_num_z=ag.pixel_num_v,
                   voxel_size_x=ag.pixel_size_h,
                   voxel_size_y=ag.pixel_size_h,
                   voxel_size_z=ag.pixel_size_v)

## FBP Reconstruction

Reconstruct the data set using the FBP processor from ASTRA

`from ccpi.astra.processors import FBP`

We Run this in the same way as the processors introduced above.

In [ ]:
#Initialise the processor
fbp = FBP(ig, ag, device='gpu')

In [ ]:
#set the input
fbp.set_input(data_processed)

In [ ]:
#Run the procesor
FBP_output = fbp.get_output()

In [ ]:
#plot the results
islicer(FBP_output, direction=0)

### Define the projector

In the 2D example we used the 2D projector from ASTRA<br>
`'AstraProjectorSimple(volume_geometry, sinogram_geometry, device)`

Use ASTRA's 3D projector, note this projector is GPU only<br>
`AstraProjector3DSimple(volume_geometry, sinogram_geometry)`

In [ ]:
# Define the projector object
print ("Define projector")
Cop = AstraProjector3DSimple(ig, ag)

### Run SIRT

In [ ]:
#setup SIRT
x_init = ig.allocate(0)
sirt = SIRT(x_init=x_init, operator=Cop, data=data_processed, update_objective_interval = 10)
sirt.max_iteration = 1000

In [ ]:
#run the algorithm
sirt.run(100, verbose = True)

In [ ]:
#plot the results
SIRT_output = sirt.get_output()
islicer(SIRT_output, direction=0)

### Run Tikhonov CGLS

In [ ]:
#define the operator
alpha = 20
L = Gradient(ig)
operator_block = BlockOperator( Cop, alpha * L, shape=(2,1))

In [ ]:
#define the data b
data_block = BlockDataContainer(data_processed, L.range_geometry().allocate(0))

In [ ]:
#setup Tikonov
x_init = ig.allocate(0)
cgls_tikhonov = CGLS(x_init=x_init, operator=operator_block, data=data_block, update_objective_interval = 10)
cgls_tikhonov.max_iteration = 1000

In [ ]:
#run the algorithm
cgls_tikhonov.run(100, verbose = True)

In [ ]:
#plot the results
CGLS_tikhonov_output = cgls_tikhonov.get_output()

islicer(CGLS_tikhonov_output, direction=0)

### Summary

In [ ]:
#compare the outputs
clim_range=(0,0.006)
slicer1=islicer(SIRT_output, direction=0,minmax=clim_range,title='SIRT')
slicer2=islicer(CGLS_tikhonov_output, direction=0,minmax=clim_range,title='CGLS')
slicer3=islicer(FBP_output, direction=0,minmax=clim_range,title='FBP')

link_islicer(slicer1,slicer2,slicer3)

In [ ]:
#compare the outputs
clim_range=(0,0.11)
slicer1=islicer(SIRT_output, direction=0,minmax=clim_range,title='SIRT')
slicer2=islicer(CGLS_tikhonov_output, direction=0,minmax=clim_range,title='CGLS')
slicer3=islicer(FBP_output, direction=0,minmax=clim_range,title='FBP')

link_islicer(slicer1,slicer2,slicer3)